# Programming of difference equations
Although the main focus of these notes is on solvers for *differential equations*, we find it useful
to include a chapter on the closely related class of problems known as *difference equations*. 
The main motivation for including this topic in a book on ODEs is to highlight the similarity 
between the two classes of problems, and in particular the similarity of the solution methods
and their implementation. Indeed, solving ODEs numerically can be seen as a two-step procedure.
First, a numerical method is applied to turn *differential equations* into *difference 
equations*, and then these equations are solved using simple for-loop. The standard formulation
of difference equations is very easy to translate into a computer program, and some readers may find
it easier to study these equations first, before moving on to ODEs. In the present chapter we will
also touch upon famous sequences and series, which have important applications both in the
numerical solution of ODEs and elsewhere. 

# Sequences and Difference Equations
Sequences is a central topic in mathematics, which has important applications in numerical analysis
and scientific computing. In the most general sense, a sequence is simply a collection of numbers:

$$
x_0,\ x_1,\ x_2,\ \ldots,\ x_n,\ldots .
$$

For some sequences we can derive a formula that gives the the $n$-th number $x_n$
as a function of $n$. For instance, all the odd numbers form a sequence

$$
1, 3, 5, 7, \dots ,
$$

and for this sequence we can write a simple formula for the $n$-th term;

$$
x_n = 2n+1 .
$$

With this formula at hand, the complete sequence can be written on a compact form;

$$
(x_n)_{n=0}^\infty,\quad x_n = 2n+1 .
$$

Other examples of sequences include

$$
\begin{align*}
& 1,\ 4,\ 9,\ 16,\ 25,\ \ldots\quad (x_n)_{n=0}^\infty,\ x_n=n^2 , \\
&1,\ {1\over 2},\ {1\over3},\ {1\over4},\ \ldots\quad (x_n)_{n=0}^\infty,\ x_n={1\over {n+1}} ,\\
&1,\ 1,\ 2,\ 6,\ 24,\ \ldots\quad (x_n)_{n=0}^\infty,\ x_n=n! ,\\
& 1,\ 1+x,\ 1+x+{1\over2}x^2,\ 1+x+{1\over2}x^2+{1\over6}x^3,\ \ldots\quad (x_n)_{n=0}^\infty,\ x_n=\sum_{j=0}^n {x^j\over j!} .
\end{align*}
$$

These are all formulated as infinite sequences, which is common in mathematics, but
in real-life applications sequences are usually finite: $(x_n)_{n=0}^N$.
Some familiar examples include the annual value of a loan or an investment.

In many cases it is impossible to derive an explicit formula for the
entire sequence, and $x_n$ is instead
given by a relation involving $x_{n-1}$ and possibly $x_{n-2}$. Such equations
are called *difference equations*, and they
can be challenging to solve with analytical methods, since in order to compute the $n$-th term of a sequence
we need to compute the entire sequence $x_0, x_1, \ldots, x_{n-1}$. This can be very tedious to do by hand or using
a calculator, but on a computer the equation is easy to solve with a for-loop. Combining sequences and difference
equations with programming therefore enables us to consider far more interesting and useful cases.

### A difference equation for computing interest.

To start with a simple example, consider the problem of computing how an invested sum of money grows over time.
In its simplest form, this problem can be written as putting $x_0$ money in a bank at year 0, with interest rate
$p$ percent per year. What is then the value after $n$ years? You may recall from earlier in IN1900
(and from high school mathematics) that the solution to this problem is given by the simple
formula
$$x_n = x_0 (1 + p/100)^n,$$
so there is really no need to formulate and solve the problem as
a difference equation. However, very simple generalizations, such as a non-constant interest rate,
makes this formula difficult to apply, while a formulation based on a
difference equation will still be applicable. To formulate the problem as a
difference equation, we use the fact that the amount $x_{n+1}$ at year $n+1$ is simply the
amount at year $n$ plus the interest for year $n$. This gives the following
relation between $x_{n+1}$ and $x_n$:

$$
x_{n+1} = x_{n} + {p\over 100}x_{n} .
$$

In order to compute $x_n$, we can now simply start with the known $x_0$, and compute $x_1, x_2, \ldots, x_n$.
The procedure involves repeating a simple calculation many times, which is tedious to do by
hand, but very well suited for a computer. The complete program for solving this
difference equation may look like:

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
x0 = 100                      # initial amount
p = 5                         # interest rate
N = 4                         # number of years
x = np.zeros(N+1)

x[0] = x0
for n in range(1,N+1):
    x[n] = x[n-1] + (p/100.0)*x[n-1]

plt.plot(x, 'ro')
plt.xlabel('years')
plt.ylabel('amount')
plt.show()

This code only involves tools that we have introduced earlier in the course. %IN1900
The three lines starting with `x[0] = x0` make up the core of the program. We
here initialize the first element in our solution array with the known `x0`, and
then step into the for-loop to compute the rest. The loop variable `n` runs from
1 to $N (=4)$, and the formula inside the loop computes `x[n]` from the known `x[n-1]`.

An alternative
formulation of the for-loop would be

```Python
        for n in range(N):
            x[n+1] = x[n] + (p/100.0)*x[n]
```

Here `n` runs from 0 to 3, and all the indices inside the loop have been increased by one so that the
end result is the same. In this case it is easy to verify that the two loops give the same result,
but mixing up the two formulations will easily lead to a loop that runs out of bounds (an `IndexError`)
or a loop where some of the sequence elements are never computed. Such mistakes are probably the
most common type of programming error when solving difference
equations, and it is a good habit to always examine the loop formulation carefully.
If an `IndexError` (or another suspected loop error)
occurs, a good debugging strategy is to look at the loop definition to find the lower and upper
value of the loop variable (here `n`), and insert both by hand into
the formulas inside the loop to check that they make sense. As an example, consider the deliberately wrong code

```Python
        for n in range(1,N+1):
            x[n+1] = x[n] + (p/100.0)*x[n]
```

Assuming that the rest of the code is unchanged from the example above,
the loop variable `n` will run from 1 to 4. If we first insert the lower bound `n=1`
into the formula, we find that the first pass of the loop
will try to compute `x[2]` from `x[1]`. However, we have only initialized `x[0]`,
so `x[1]` is zero, and therefore `x[2]` and all subsequent values will be
set to zero. Furthermore, if we insert the upper bound `n=4` we see that the formula will
try to access `x[5]`, but this does not exist and we get an `IndexError`.
Performing such simple analysis of a for-loop is often a good way to reveal the source of the error
and give an idea of how it can be fixed.


### Solving a difference equation without using arrays.

The program above stored the sequence as an array,
which is a convenient way to program the solver and enables us to plot the
entire sequence. However, if we are only interested in the solution at a single
point, i.e., $x_n$, there is no need to store the entire sequence. Since
each $x_n$ only depends on the previous value $x_{n-1}$, we only need to
store the last two values in memory. A complete loop can look like this:

In [2]:
x_old = x0
for n in index_set[1:]:
    x_new = x_old + (p/100.)*x_old
    x_old = x_new  # x_new becomes x_old at next step
print('Final amount: ', x_new)

For this simple case we can actually make the code even shorter, since `x_old` is only used
in a single line and we can just as well overwrite it once it has been used:

In [3]:
x = x0			#x is here a single number, not array
for n in index_set[1:]:
    x = x + (p/100.)*x
print('Final amount: ', x)

We see that these codes store just one or two numbers, and for each pass through the loop we simply update these and overwrite
the values we no longer need. Although this approach is quite simple, and we obviously save some memory since we do not store the
complete array, programming with an array `x[n]` is usually safer, and we are often interested in plotting the entire sequence.
We will therefore mostly use arrays in the subsequent examples.

### Extending the solver for the growth of money.

Say we are interested in changing our model for interest rate, to a model
where the interest is added every day instead of every year. The interest rate per day is $r=p/D$ if $p$ is the
annual interest rate and $D$ is the number of days in a year. A common model in business applies $D=360$,
but $n$ counts exact (all) days. The difference equation relating one day's amount to the previous is the same as above

$$
x_{n} = x_{n-1} + {r\over 100}x_{n-1} ,
$$

except that the yearly interest rate has been replaced by the daily ($r$). If we are interested in how much the money grows from
a given date to another we also need to find the number of days between those dates. This calculation could of course be done
by hand, but Python has a convenient module named `datetime` for this purpose. The following session
illustrates how it can be used:

In [4]:
import datetime
date1 = datetime.date(2017, 9, 29)  # Sep 29, 2017
date2 = datetime.date(2018, 8, 4)  # Aug 4, 2018
diff = date2 - date1
print(diff.days)

Putting these tools together, a complete program for daily interest rates may look like

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

x0 = 100                           # initial amount
p = 5                              # annual interest rate
r = p/360.0                        # daily interest rate

date1 = datetime.date(2017, 9, 29)
date2 = datetime.date(2018, 8, 4)
diff = date2 - date1
N = diff.days
index_set = range(N+1)
x = np.zeros(len(index_set))

x[0] = x0
for n in index_set[1:]:
    x[n] = x[n-1] + (r/100.0)*x[n-1]

plt.plot(index_set, x)
plt.xlabel('days')
plt.ylabel('amount')
plt.show()

This program is slightly more sophisticated than the first one, but one may still argue that solving this problem with a difference
equation is unnecessarily complex, since we could just apply
the well-known formula $x_n =x_0(1+\frac{r}{100})^n$
to compute any $x_n$ we want. However, we know that interest rates change quite often, and
the formula is only valid for a constant $r$. For the program based on solving the
difference equation, on the other hand, only minor changes are
needed in the program to handle a varying interest rate.
The simplest approach is to let `p` be an array of the same length as the number of days, and fill
it with the correct interest rates for each day. The modifications to the program above may look like this:

```Python
        p = np.zeros(len(index_set))
        # fill p[n] with correct values
        
        r = p/360.0                      # daily interest rate
        x = np.zeros(len(index_set))
        
        x[0] = x0
        for n in index_set[1:]:
            x[n] = x[n-1] + (r[n-1]/100.0)*x[n-1]
```

The only real difference from the previous example is that we initialize `p` as an array,
and then `r = p/360.0` becomes an array of the same length. In
the formula inside the for-loop we then look up the correct value `r[n-1]` for
each iteration  of the loop. Filling `p`
with the correct values can be non-trivial, but many cases can be
handled quite easily. For instance, say the interest rate
is piecewise constant and increases from 4.0% to 5.0% on a given date.
Code for filling the array with values may then
look like this

```Python
        date0 = datetime.date(2017, 9, 29)
        date1 = datetime.date(2018, 2, 6)
        date2 = datetime.date(2018, 8, 4)
        Np = (date1-date0).days
        N = (date2-date0).days
        
        p = np.zeros(len(index_set))
        p[:Np] = 4.0
        p[Np:] = 5.0
```

# More Examples of Difference Equations
As noted above, sequences, series, and difference equations have countless applications
in mathematics, science, and engineering. Here we present a selection of well known examples.

### Fibonacci numbers as a difference equation

The sequence defined by the difference equation

$$
x_n = x_{n-1} + x_{n-2},\quad x_0=1,\ x_1=1 .
$$

is called the Fibonacci numbers. It was originally derived for modeling rat populations,
but it has a range of
interesting mathematical properties and has therefore attracted much attention from mathematicians.
The equation for the Fibonacci numbers differs from the previous ones,
since $x_n$ depends on the two previous values ($n-1$, $n-2$), which
makes this a *second order difference equation*. This classification is important for mathematical
solution techniques, but in a program the difference between first and second order equations is small.
A complete code to solve the difference equation and generate the Fibonacci numbers can be written as

In [6]:
import sys
from numpy import zeros

N = int(sys.argv[1])
x = zeros(N+1, int)
x[0] = 1
x[1] = 1
for n in range(2, N+1):
    x[n] = x[n-1] + x[n-2]
    print(n, x[n])

Notice that in this case we need to initialize both `x[0]` and `x[1]` before starting
the loop, since the update formula involves both `x[n-1]` and `x[n-2]`. This is the
main difference between this second order equation and the programs for first order
equations considered above.
The Fibonacci numbers grow quickly and running this program for large $N$ will
lead to problems with overflow (try for instance $N = 100$).
The NumPy `int` type supports up to 9223372036854775807, which is almost
$10^{19}$, so this is very rarely a problem in practical applications.
We can fix the problem by avoiding NumPy arrays and instead use
the standard Python `int` type, but we will not go into these details here.

### Logistic growth

If we return to the initial problem of calculating growth of money in a bank, we can write the
classical solution formula more compactly as

$$
x_n = x_0 (1 + p/100)^n = x_0C^n \quad (= x_0e^{n\ln C}) ,
$$

with $C=(1+p/100)$.
Since $n$ counts years, this is an example of exponential growth in time, with
the general formula $x= x_0e^{\lambda t}$.
Populations of humans, animals, and other organisms also exhibit the same type
of growth when there are unlimited resources
(space and food), and the model for exponential growth therefore has many applications in biology.
However, most environments can only support a finite number $R$ of individuals, while in the exponential growth model
the population will continue to grow indefinitely. How can we alter the 
equation to be a more realistic model for growing populations?


Initially, when resources are abundant, we want the growth to be exponential,
i.e., to grow with a given rate $r$\% per year according to the difference
equation introduced above:

$$
x_n = x_{n-1} + (r/100) x_{n-1} .
$$

To enforce the growth limit as $x_n \rightarrow R$,
$r$ must decay to zero as $x_n$ approaches $R$. The simplest
variation of $r(n)$ is linear:

$$
r(n) = \varrho \left(1 - {x_n\over R}\right)
$$

We observe that $r(n)\approx \varrho$ for small $n$, when $x_n\ll R$, and
$r(n) \rightarrow 0$ as $n$ grows and $x_n\rightarrow R$. This formulation of
the growth rate leads to the logistic growth model:

$$
x_n = x_{n-1} + \frac{\varrho}{100} x_{n-1}\left(1 - {x_{n-1}\over R}\right) .
$$

This is a *nonlinear* difference equation, while all the examples considered above were linear.
The distinction between linear and non-linear equations is very important for
mathematical analysis of the equations, but it does not make much difference when solving the equation
in a program. To modify the interest rate program above to describe logistic growth, we can simply replace
the line

```Python
        x[n] = x[n-1] + (p/100.0)*x[n-1]
```

by

```Python
        x[n] = x[n-1] + (rho/100)*x[n-1]*(1 - x[n-1]/R)
```

A complete program may look like

In [7]:
import numpy as np
import matplotlib.pyplot as plt
x0 = 100                      # initial population
rho = 5                       # growth rate in %
R = 500                       # max population (carrying capacity)
N = 200                       # number of years

index_set = range(N+1)
x = np.zeros(len(index_set))

x[0] = x0
for n in index_set[1:]:
    x[n] = x[n-1] + (rho/100) *x[n-1]*(1 - x[n-1]/R)

plt.plot(index_set, x)
plt.xlabel('years')
plt.ylabel('amount')
plt.show()

Note that the logistic growth model is more commonly formulated as an
ordinary differential equation (ODE), and we will consider this formulation in
the next chapter. For certain choices of numerical method and discretization
parameters, the program for solving the ODE is identical to the program
for the difference equation considered here.


### The factorial as a difference equation.

The factorial $n!$ is defined as

<!-- Equation labels as ordinary links -->
<div id="factorial"></div>

$$
\begin{equation}
n! = n(n-1)(n-2)\cdots 1,\quad 0!=1
\label{factorial} \tag{1}
\end{equation}
$$

The following difference equation has $x_n=n!$ as solution and can be used to compute the factorial:

$$
x_n = nx_{n-1},\quad x_0 = 1
$$

As above, a natural question to ask is whether such a difference equation is
very useful, when we can simply use the formula ([1](#factorial)) to compute
the factorial for any value of $n$. One answer to this question is that in
many applications, some of which will be considered below,
we need to compute the
entire sequence of factorials $x_n=n!$ for $n=0, \ldots N$.
We could still apply ([1](#factorial)) to compute each one, but it
involves a lot of redundant computations, since we perform $n$ multiplications
for each new $x_n$. When solving the difference equation, each
new $x_n$ requires only a single multiplication, and for large values of $n$
this may speed up the program considerably.


### Newton's method as a difference equation

Newton's method is a popular method for solving non-linear equations on the form

$$
f(x)=0 .
$$

Starting from some initial guess $x_0$, Newton's method
gradually improves the approximation by iterations

$$
x_n = x_{n-1} - {f(x_{n-1})\over f'(x_{n-1})} .
$$

We may now recognize this as nonlinear first-order difference equation.
As $n\rightarrow\infty$, we hope that $x_n\rightarrow x_s$, where $x_s$ is
the solution to $f(x_s)=0$. In practice we solve the equation for $n\leq N$,
for some finite $N$, just as for the difference equations considered earlier.
But how do we choose $N$ so that $x_N$ is sufficiently close to the true solution
$x_s$? Since we want to solve $f(x)=0$, the best approach is to solve the equation
until $f(x)\leq \epsilon$, where $\epsilon$ is a small tolerance. In practice, Newton's
method will usually converge rather quickly, or not converge at all, so setting
some upper bound on the number of iterations
is a good idea. A simple implementation of Newton's method as a Python function
may look like

In [8]:
def Newton(f, dfdx, x, epsilon=1.0E-7, max_n=100):
    n = 0
    while abs(f(x)) > epsilon and n <= max_n:
        x = x - f(x)/dfdx(x)
        n += 1
    return x, n, f(x)

The arguments to the function are Python functions `f` and `dfdx` implementing
$f(x)$ and its derivative. Both of these arguments are called inside the function
and must therefore be callable. The `x` argument is the initial guess for the solution $x$,
and the two optional arguments at the end are the tolerance and the maximum
number of iteration. Although the method is implemented as a while-loop rather
than a for-loop, the main structure of the algorithm is exactly the same as for the
other difference equations considered earlier.



# Systems of Difference Equations
All the examples considered so far have been scalar difference equations, which describe how a single quantity
changes from one step to the next. However, many applications require tracking multiple variables
simultaneously, and dynamics of these variables may be coupled in the sense that the value of one variable
at step $n$ depends on the value of multiple variables at step $n-1$. As an example we may consider a simple
extension of the interest rate model considered earlier. Assume that we have a fortune $F$ invested with an
annual interest rate of $p$ percent, just as above, but now we also want to consume an amount $c_n$ every year.
We model for computing our fortune $x_n$ at year $n$ can be 
formulated as a small extension of the difference equation considered earlier.
First, simple reasoning tells us that the fortune at year $n$ is equal to
the fortune at year $n-1$ plus the interest minus the amount we
spent in year $n-1$. We have

$$
x_n = x_{n-1} + {p\over 100}x_{n-1} - c_{n-1} .
$$

In the simplest case we can assume that $c_n$ is constant, which would make this model a trivial
extension of the interest rate model considered earlier. However, it is more natural to let $c_n$ increase because of
inflation, and in this case we obtain a system of difference equations describing the evolution of $x_n$ and $c_n$.
For instance, we may assume that $c_n$ should grow with a rate of $I$ percent per year, and in the first year
we want to consume $q$ percent of first year's interest. The governing system of difference equations then
becomes

$$
\begin{align*}
x_n &= x_{n-1} + {p\over 100}x_{n-1} - c_{n-1}, \\
c_n &= c_{n-1} + {I\over 100}c_{n-1} .
\end{align*}
$$

with initial conditions $x_0=F$ and $c_0 = (pF/100)(q/100)  = \frac{pFq}{10000}$. This is a coupled
system of two first order difference equations, but the programming is not much more difficult than for the
single equation above. We simply create two arrays `x` and `c`, initialize `x[0]`
and `c[0]` to the given initial conditions, and then update `x[n]` and `c[n]`
inside the loop. A complete code may look like

In [9]:
import numpy as np
import matplotlib.pyplot as plt
F = 1e7                      # initial amount
p = 5                         # interest rate
I = 3
q = 75
N = 40                         # number of years
index_set = range(N+1)
x = np.zeros(len(index_set))
c = np.zeros_like(x)

x[0] = F
c[0] = q*p*F*1e-4

for n in index_set[1:]:
    x[n] = x[n-1] + (p/100.0)*x[n-1] - c[n-1]
    c[n] = c[n-1] + (I/100.0)*c[n-1]

plt.plot(index_set, x, 'ro',label = 'Fortune')
plt.plot(index_set, c, 'go', label = 'Yearly consume')
plt.xlabel('years')
plt.ylabel('amounts')
plt.legend()
plt.show()

As another example of a system of difference equations, we may consider an extension of the
logistic growth model considered earlier. While the logistic model describes the growth
of a single population in the absence of predators, the famous Lotke-Volterra model
describes the interaction of two species, a predator and a prey, in the same ecosystem.
If we let $x_n$ be the number of prey and $y_n$ the number of predators on day $n$,
the model for the population dynamics can be written as

$$
\begin{align*}
x_n &= x_{n-1} + a x_{n-1} − b x_{n-1}y_{n-1}, \\
y_n &= y_{n-1} + db x_{n-1}y_{n-1} − c y_{n-1}.
\end{align*}
$$

Here, $a$ is the natural growth rate of the prey in the absence of predators,
$b$ is the death rate of prey per encounter of prey and predator, 
$c$ is the natural death rate of predators in the absence of food (prey), and
$d$ is the efficiency of turning predated prey into predators. This is a system
of two first-order difference equations, just as the previous example, and a complete
solution code may look as follows.

In [10]:
import numpy as np
import matplotlib.pyplot as plt
x0 = 100                # initial prey population
y0 = 8                  # initial predator pop.
a = 0.0015
b = 0.0003
c = 0.006
d = 0.5
N = 10000               # number of time units (days)
index_set = range(N+1)
x = np.zeros(len(index_set))
y = np.zeros_like(x)

print(x.shape)

x[0] = x0
y[0] = y0

for n in index_set[1:]:
    x[n] = x[n-1] + a*x[n-1] - b*x[n-1]*y[n-1]
    y[n] = y[n-1] + d*b*x[n-1]*y[n-1] - c*y[n-1]

plt.plot(index_set, x,label = 'Prey')
plt.plot(index_set, y, label = 'Predator')
plt.xlabel('Time')
plt.ylabel('Population')
plt.legend()
plt.show()

We 





# Taylor Series and Approximations
One extremely important use of sequences and series is for approximating
functions. For instance, commonly used functions such as
$\sin x, \ln x$, and $e^x$ have been defined to have some desired mathematical
properties, and we have an intuitive understanding of how they look,
but we need some kind of algorithm to evaluate the function values.
A convenient approach would be to approximate these functions by polynomials,
since they are easy to calculate. It turns out that such polynomial
approximations exist, and they have been used for centuries to compute
exponentials, trigonometric, and other functions. By far, the most famous and
widely used series of this kind are the Taylor series, discovered in 1715:

<!-- Equation labels as ordinary links -->
<div id="taylor0"></div>

$$
\begin{equation}
f(x) = \sum_{k=0}^\infty \frac{1}{k!}(\frac{d^kf(0)}{dx^k})x^k .
\label{taylor0} \tag{2}\end{equation}
$$

Here, the notation $d^kf(0)/dx^k$ means the $k$-th derivative of $f$ evaluated
at $x=0$. We can calculate a few of the terms in the sum to get

$$
f(x) = f(0)+f'(0)x+\frac{1}{2}f''(0)x^2 + \frac{1}{6} f'''(0) x^3 \ldots ,
$$

which makes it obvious that the right hand side of ([2](#taylor0)) is in fact a polynomial in $x$.
Taylor's result means that for any function $f(x)$, if we can compute the
function value and its derivatives for $x=0$, we can approximate the function
value at any $x$ by evaluating a polynomial. For practical applications, we always work
with a truncated version of the Taylor series:

<!-- Equation labels as ordinary links -->
<div id="taylor1"></div>

$$
\begin{equation}
f(x) \approx \sum_{k=0}^{N}
\frac{1}{k!}(\frac{d^kf(0)}{dx^k} )x^k .
\label{taylor1} \tag{3}\end{equation}
$$

The approximation improves as $N$ is increased, but the most popular
choice is actually $N=1$, which gives a reasonable approximation close to $x=0$ and
has been essential in developing physics and technology. By a shift of variables
we can make these truncated Taylor series accurate around any value $x=a$:

$$
f(x) \approx \sum_{k=0}^{N}
\frac{1}{k!}(\frac{d^kf(a)}{dx^k} )(x-a)^k .
$$

One of many applications of truncated Taylor series is to derive numerical methods for ODEs, and to analyze
their accuracy, as we briefly introduced in Chapter 2.

As a specific example of a Taylor series, consider the exponential function, where
we know that $d^k e^x/dx^k = e^x$ for all $k$, and $e^0 = 1$. 
Inserting this into ([3](#taylor1)) yields

$$
\begin{align*}
e^x &= \sum_{k=0}^\infty \frac{x^k}{k!} \\
    &\approx \sum_{k=0}^N \frac{x^k}{k!} .
\end{align*}
$$

Choosing, for instance, $N=1$ and $N=4$, we get

$$
\begin{align*}
e^x&\approx 1 + x , \\
e^x&\approx 1 + x + \frac{1}{2}x^2 + \frac{1}{6}x^3,
\end{align*}
$$

respectively.
These approximations are obviously not very accurate for large $x$,
but close to $x=0$ they are sufficiently accurate for many
applications. Taylor series approximations for other functions can be constructed by similar arguments.
Consider, for instance, $sin(x)$, where the derivatives follow the repetitive pattern
$sin'(x) = cos(x), sin''(x) = -sin(x), sin'''(x) = -cos(x), \dots...$. We also have $sin(0)=0, cos(0)=1$,
so in general we have 
$d^k sin(0)/dx^k = (-1)^k mod(k,2)$, where $mod(k,2)$ is zero for $k$ even and
<!-- one for $k$ odd. Inserting this expression into ([2](#taylor0)), we get -->

$$
\sin x = \sum_{k=0}^\infty (-1)^k\frac{x^{2k+1}}{(2k+1)!} .
$$

### Taylor series formulated as difference equations.

We consider again the Taylor series for $e^x$ around $x=0$, given by

$$
e^x= \sum_{k=0}^\infty {x^k\over k!} .
$$

If we now define $e_n$ as the approximation with $n$ terms, i.e. for $k=0,\ldots, n-1$,
we have

$$
e_{n}=\sum_{k=0}^{n-1} \frac{x^k}{k!} =
\sum_{k=0}^{n-2} \frac{x^k}{k!} + \frac{x^{n-1}}{(n-1)!},
$$

and we can formulate the sum in $e_n$ as the difference equation

<!-- Equation labels as ordinary links -->
<div id="taylor_diffeq0"></div>

$$
\begin{equation}
e_n = e_{n-1} + \frac{x^{n-1}}{(n-1)!},\quad e_0=0 .
\label{taylor_diffeq0} \tag{4}
\end{equation}
$$

We see that this difference equation involves $(n-1)!$, and computing
the complete factorial for every iteration involves a large number of redundant
multiplications. Above we introduced a difference equation for the factorial, and
this idea can be utilized to formulate a more efficient computation of the
Taylor polynomial. We have that

$$
\frac{x^n}{n!} = \frac{x^{n-1}}{(n-1)!}\cdot \frac{x}{n},
$$

and if we let $a_n = x^n/n!$ it can be computed efficiently by solving

$$
a_n = a_{n-1}\frac{x}{n},\quad a_0 = 1 .
$$

Now we can formulate a system of two difference equations for the Taylor polynomial, where we update each term via
the $a_n$ equation and sum the terms via the $e_n$ equation:

$$
\begin{align*}
e_n &= e_{n-1} + a_{n-1},\quad e_0 = 0, \\
a_n &= \frac{x}{n} a_{n-1} , \quad a_0 = 1.
\end{align*}
$$

Although we are here solving a system of two difference equations, the computation
is far more efficient than solving the single equation in ([4](#taylor_diffeq0))
directly, since we avoid the repeated multiplications involved in the factorial computation.

A complete Python code for solving the system of difference equation and computing the Taylor
approximation to the exponential function may look like

In [11]:
import numpy as np

x = 0.5 #approximate exp(x) for x = 0.5

N = 5
index_set = range(N+1)
a = np.zeros(len(index_set))
e = np.zeros(len(index_set))
a[0] = 1

print(f'Exact: exp({x}) = {np.exp(x)}')
for n in index_set[1:]:
    e[n] = e[n-1] + a[n-1]
    a[n] = x/n*a[n-1]
    print(f'n = {n}, approx. {e[n]}, error = {np.abs(e[n]-np.exp(x)):4.5f}')

This small program first prints the exact value $e^x$ for $x=0.5$, and then
the Taylor approximation and associated error for $n=1$ to $n=5$. The Taylor series
approximation is most accurate close to $x=0$, so choosing a larger value
of $x$ leads to larger errors, and we need to also increase $n$ for the approximation
to be accurate.